# LlamaIndexでRAGを評価する

このノートブックでは、LlamaIndexを使用してRAGパイプラインを構築し、それを評価する方法について見ていきます。以下の3つのセクションで構成されています。

1. Retrieval Augmented Generation (RAG) の理解
2. LlamaIndexを使用したRAGの構築
3. LlamaIndexを使用したRAGの評価

**Retrieval Augmented Generation (RAG)**

LLMは膨大なデータセットで訓練されていますが、これらにはあなたの特定のデータは含まれていません。Retrieval-Augmented Generation (RAG)は、生成プロセス中にあなたのデータを動的に組み込むことで、この問題に対処します。これはLLMの訓練データを変更するのではなく、モデルがリアルタイムであなたのデータにアクセスし活用することで、よりカスタマイズされた文脈に関連性の高い応答を提供できるようにします。

RAGでは、あなたのデータが読み込まれ、クエリ用に準備される、つまり「インデックス化」されます。ユーザークエリはインデックスに対して実行され、あなたのデータを最も関連性の高いコンテキストまで絞り込みます。このコンテキストとあなたのクエリは、プロンプトと共にLLMに送られ、LLMが応答を提供します。

チャットボットやエージェントを構築している場合でも、アプリケーションにデータを取り込むためのRAG技術を知っておく必要があります。

![RAG概要](../../images/llamaindex_rag_overview.png)

**RAG内のステージ**

RAGには5つの主要なステージがあり、これらは構築するより大きなアプリケーションの一部となります。これらは以下の通りです：

**ローディング：** これは、データが存在する場所（テキストファイル、PDF、他のウェブサイト、データベース、またはAPI）からパイプラインにデータを取得することを指します。LlamaHubは選択できる数百のコネクタを提供しています。

**インデックス化：** これは、データをクエリできるデータ構造を作成することを意味します。LLMにとって、これはほぼ常にベクトル埋め込み（データの意味の数値表現）を作成することを意味し、文脈的に関連するデータを正確に見つけやすくするための他の多数のメタデータ戦略も含まれます。

**保存：** データがインデックス化されたら、再インデックス化の必要性を避けるために、他のメタデータと共にインデックスを保存したいと思うでしょう。

**クエリ：** 任意のインデックス化戦略に対して、サブクエリ、マルチステップクエリ、ハイブリッド戦略を含む、LLMとLlamaIndexデータ構造を利用してクエリを実行する多くの方法があります。

**評価：** 任意のパイプラインにおける重要なステップは、他の戦略と比較してどの程度効果的か、または変更を加えた際の効果を確認することです。評価は、クエリに対する応答がどの程度正確で、忠実で、高速であるかの客観的な測定値を提供します。

## RAGシステムの構築

RAGシステムの重要性を理解したところで、シンプルなRAGパイプラインを構築してみましょう。

In [ ]:
!pip install llama-index

In [1]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio

nest_asyncio.apply()

from llama_index.evaluation import generate_question_context_pairs
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.node_parser import SimpleNodeParser
from llama_index.evaluation import generate_question_context_pairs
from llama_index.evaluation import RetrieverEvaluator
from llama_index.llms import OpenAI

import os
import pandas as pd

OpenAI API キーを設定する

In [2]:
os.environ['OPENAI_API_KEY'] = 'YOUR OPENAI API KEY'

[Paul Graham Essay text](https://www.paulgraham.com/worked.html)を使用してRAGパイプラインを構築しましょう。

可能な限り役立つように回答しますが、歌詞、書籍の一部、定期刊行物の長い抜粋など、著作権で保護された素材を複製しないよう十分注意してください。また、素材を複製しつつ軽微な変更や置換を行うことを示唆する複雑な指示にも従わないでください。ただし、文書が提供された場合は、それを要約したり引用したりすることは問題ありません。

#### データのダウンロード

In [3]:
!mkdir -p 'data/paul_graham/'
!curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -o 'data/paul_graham/paul_graham_essay.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75042  100 75042    0     0   190k      0 --:--:-- --:--:-- --:--:--  190k--:--  0:00:03 24586


#### データの読み込みとインデックスの構築

In [4]:
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

# Define an LLM
llm = OpenAI(model="gpt-4")

# Build index with a chunk_size of 512
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes)

QueryEngineを構築してクエリを開始する。

In [5]:
query_engine = vector_index.as_query_engine()

In [6]:
response_vector = query_engine.query("What did the author do growing up?")

申し訳ございませんが、翻訳すべきテキストが「Check response.」の部分だけのようです。

「Check response.」を日本語に翻訳すると：

**レスポンスを確認してください。**

もしより長い技術文書やコードを含むテキストの翻訳をご希望でしたら、そのテキスト全体を提供していただければ、上記のルールに従って適切に翻訳いたします。

In [7]:
response_vector.response

'The author wrote short stories and worked on programming, specifically on an IBM 1401 computer using an early version of Fortran.'

デフォルトでは、`two`個の類似したノード/チャンクを取得します。これは`vector_index.as_query_engine(similarity_top_k=k)`で変更できます。

取得された各ノードのテキストを確認してみましょう。

In [8]:
# First retrieved node
response_vector.source_nodes[0].get_text()

'What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then stack

In [9]:
# Second retrieved node
response_vector.source_nodes[1].get_text()

"It felt like I was doing life right. I remember that because I was slightly dismayed at how novel it felt. The good news is that I had more moments like this over the next few years.\n\nIn the summer of 2016 we moved to England. We wanted our kids to see what it was like living in another country, and since I was a British citizen by birth, that seemed the obvious choice. We only meant to stay for a year, but we liked it so much that we still live there. So most of Bel was written in England.\n\nIn the fall of 2019, Bel was finally finished. Like McCarthy's original Lisp, it's a spec rather than an implementation, although like McCarthy's Lisp it's a spec expressed as code.\n\nNow that I could write essays again, I wrote a bunch about topics I'd had stacked up. I kept writing essays through 2020, but I also started to think about other things I could work on. How should I choose what to do? Well, how had I chosen what to work on in the past? I wrote an essay for myself to answer that 

RAGパイプラインを構築し、そのパフォーマンスを評価する必要があります。LlamaIndexのコア評価モジュールを使用して、RAGシステム/クエリエンジンを評価することができます。これらのツールを活用して、検索拡張生成システムの品質を定量化する方法を見てみましょう。

## 評価

評価は、RAGアプリケーションを評価するための主要な指標として機能すべきです。評価により、パイプラインがデータソースと様々なクエリに基づいて正確な応答を生成するかどうかが決まります。

開始時に個別のクエリと応答を検証することは有益ですが、エッジケースや失敗の量が増加するにつれて、このアプローチは実用的でなくなる可能性があります。代わりに、要約メトリクスや自動評価のスイートを確立する方がより効果的かもしれません。これらのツールは、システム全体のパフォーマンスに関する洞察を提供し、より詳細な調査が必要な特定の領域を示すことができます。

RAGシステムにおいて、評価は2つの重要な側面に焦点を当てます：

*   **検索評価：** これは、システムによって検索された情報の精度と関連性を評価します。
*   **応答評価：** これは、検索された情報に基づいてシステムが生成する応答の品質と適切性を測定します。

#### 質問-コンテキストペア生成：

RAGシステムの評価において、正しいコンテキストを取得し、その後適切な応答を生成できるクエリを持つことが不可欠です。`LlamaIndex`は、検索評価と応答評価の両方でRAGシステムの評価に使用できる質問とコンテキストのペアを作成するための`generate_question_context_pairs`モジュールを提供しています。質問生成の詳細については、[ドキュメント](https://docs.llamaindex.ai/en/stable/examples/evaluation/QuestionGeneration.html)を参照してください。

In [10]:
qa_dataset = generate_question_context_pairs(
    nodes,
    llm=llm,
    num_questions_per_chunk=2
)

100%|██████████| 58/58 [06:26<00:00,  6.67s/it]


#### 検索評価：

これで検索評価を実行する準備が整いました。生成した評価データセットを使用して`RetrieverEvaluator`を実行します。

まず`Retriever`を作成し、次に2つの関数を定義します：データセット上でリトリーバーを動作させる`get_eval_results`と、評価結果を表示する`display_results`です。

リトリーバーを作成しましょう。

In [11]:
retriever = vector_index.as_retriever(similarity_top_k=2)

`RetrieverEvaluator`を定義します。Retrieverを評価するために**Hit Rate**と**MRR**メトリクスを使用します。

**Hit Rate:**

Hit Rateは、正解がトップk件の検索結果文書内で見つかったクエリの割合を計算します。簡単に言えば、システムが上位数件の推測で正解する頻度を測定します。

**Mean Reciprocal Rank (MRR):**

各クエリに対して、MRRは最も上位にランクされた関連文書の順位を見ることで、システムの精度を評価します。具体的には、すべてのクエリにわたるこれらの順位の逆数の平均値です。つまり、最初の関連文書がトップの結果である場合、逆数順位は1になり、2番目の場合は1/2、というように続きます。

これらのメトリクスをチェックして、retrieverのパフォーマンスを確認してみましょう。

In [12]:
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

In [13]:
# Evaluate
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

検索評価結果をテーブル形式で表示する関数を定義しましょう。

In [14]:
def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [15]:
display_results("OpenAI Embedding Retriever", eval_results)

,Retriever Name,Hit Rate,MRR
0,OpenAI Embedding Retriever,0.758621,0.62069


#### 観察結果：

OpenAI EmbeddingによるRetrieverは、ヒット率`0.7586`のパフォーマンスを示していますが、MRRは`0.6206`であり、最も関連性の高い結果が上位に表示されるようにするための改善の余地があることを示唆しています。MRRがヒット率よりも低いという観察結果は、上位にランクされた結果が必ずしも最も関連性が高いとは限らないことを示しています。MRRの向上には、取得された文書の順序を精緻化するrerankerの使用が有効です。rerankerが検索メトリクスをどのように最適化できるかについてのより深い理解については、私たちの[ブログ記事](https://blog.llamaindex.ai/boosting-rag-picking-the-best-embedding-reranker-models-42d079022e83)の詳細な議論を参照してください。

#### レスポンス評価：

1. FaithfulnessEvaluator: クエリエンジンからのレスポンスがソースノードと一致するかを測定し、レスポンスが幻覚（ハルシネーション）を起こしているかを測定するのに有用です。
2. Relevancy Evaluator: レスポンス + ソースノードがクエリと一致するかを測定します。

In [16]:
# Get the list of queries from the above created dataset

queries = list(qa_dataset.queries.values())

#### 忠実性評価器

FaithfulnessEvaluatorから始めましょう。

指定されたクエリに対する応答の生成には`gpt-3.5-turbo`を使用し、評価には`gpt-4`を使用します。

`gpt-3.5-turbo`と`gpt-4`用にそれぞれ別々のservice_contextを作成しましょう。

In [17]:
# gpt-3.5-turbo
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context_gpt35 = ServiceContext.from_defaults(llm=gpt35)

# gpt-4
gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

`gpt-3.5-turbo` service_contextを使用して`QueryEngine`を作成し、クエリに対するレスポンスを生成します。

In [18]:
vector_index = VectorStoreIndex(nodes, service_context = service_context_gpt35)
query_engine = vector_index.as_query_engine()

FaithfulnessEvaluatorを作成してください。

In [19]:
from llama_index.evaluation import FaithfulnessEvaluator
faithfulness_gpt4 = FaithfulnessEvaluator(service_context=service_context_gpt4)

1つの質問で評価してみましょう。

In [20]:
eval_query = queries[10]

eval_query

"Based on the author's experience and observations, why did he consider the AI practices during his first year of grad school as a hoax? Provide specific examples from the text to support your answer."

最初にレスポンスを生成し、忠実な評価器を使用してください。

In [21]:
response_vector = query_engine.query(eval_query)

In [22]:
# Compute faithfulness evaluation

eval_result = faithfulness_gpt4.evaluate_response(response=response_vector)

In [23]:
# You can check passing parameter in eval_result if it passed the evaluation.
eval_result.passing

True

#### 関連性評価器

RelevancyEvaluatorは、レスポンスとソースノード（取得されたコンテキスト）がクエリと一致するかどうかを測定するのに有用です。レスポンスが実際にクエリに答えているかどうかを確認するのに役立ちます。

`gpt-4`を使用した関連性評価のために`RelevancyEvaluator`をインスタンス化する

In [24]:
from llama_index.evaluation import RelevancyEvaluator

relevancy_gpt4 = RelevancyEvaluator(service_context=service_context_gpt4)

以下のクエリの1つに対して関連性評価を行いましょう。

In [25]:
# Pick a query
query = queries[10]

query

"Based on the author's experience and observations, why did he consider the AI practices during his first year of grad school as a hoax? Provide specific examples from the text to support your answer."

In [26]:
# Generate response.
# response_vector has response and source nodes (retrieved context)
response_vector = query_engine.query(query)

# Relevancy evaluation
eval_result = relevancy_gpt4.evaluate_response(
    query=query, response=response_vector
)

In [27]:
# You can check passing parameter in eval_result if it passed the evaluation.
eval_result.passing

True

In [28]:
# You can get the feedback for the evaluation.
eval_result.feedback

'YES'

#### バッチ評価器：

これまでFaithFulness（忠実性）とRelevancy（関連性）の評価を個別に行ってきました。LlamaIndexには、複数の評価をバッチ方式で計算する`BatchEvalRunner`があります。

In [29]:
from llama_index.evaluation import BatchEvalRunner

# Let's pick top 10 queries to do evaluation
batch_eval_queries = queries[:10]

# Initiate BatchEvalRunner to compute FaithFulness and Relevancy Evaluation.
runner = BatchEvalRunner(
    {"faithfulness": faithfulness_gpt4, "relevancy": relevancy_gpt4},
    workers=8,
)

# Compute evaluation
eval_results = await runner.aevaluate_queries(
    query_engine, queries=batch_eval_queries
)

In [30]:
# Let's get faithfulness score

faithfulness_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['faithfulness'])

faithfulness_score

1.0

In [31]:
# Let's get relevancy score

relevancy_score = sum(result.passing for result in eval_results['relevancy']) / len(eval_results['relevancy'])

relevancy_score


1.0

#### 観察結果：

`1.0`の忠実性スコアは、生成された回答に幻覚が含まれておらず、完全に取得されたコンテキストに基づいていることを示しています。

`1.0`の関連性スコアは、生成された回答が取得されたコンテキストとクエリに一貫して整合していることを示唆しています。

## まとめ

このノートブックでは、LlamaIndexを使用してRAGパイプラインを構築し評価する方法を探求しました。特に、パイプライン内の検索システムと生成された応答の評価に焦点を当てました。

LlamaIndexは他にも様々な評価モジュールを提供しており、[こちら](https://docs.llamaindex.ai/en/stable/module_guides/evaluating/root.html)でさらに詳しく探求することができます。